--- Day 8: Treetop Tree House ---

The expedition comes across a peculiar patch of tall trees all planted carefully in a grid. The Elves explain that a previous expedition planted these trees as a reforestation effort. Now, they're curious if this would be a good location for a tree house.

First, determine whether there is enough tree cover here to keep a tree house hidden. To do this, you need to count the number of trees that are visible from outside the grid when looking directly along a row or column.

The Elves have already launched a quadcopter to generate a map with the height of each tree (your puzzle input). For example:
```
30373
25512
65332
33549
35390
```
Each tree is represented as a single digit whose value is its height, where 0 is the shortest and 9 is the tallest.

A tree is visible if all of the other trees between it and an edge of the grid are shorter than it. Only consider trees in the same row or column; that is, only look up, down, left, or right from any given tree.

All of the trees around the edge of the grid are visible - since they are already on the edge, there are no trees to block the view. In this example, that only leaves the interior nine trees to consider:

The top-left 5 is visible from the left and top. (It isn't visible from the right or bottom since other trees of height 5 are in the way.)
The top-middle 5 is visible from the top and right.
The top-right 1 is not visible from any direction; for it to be visible, there would need to only be trees of height 0 between it and an edge.
The left-middle 5 is visible, but only from the right.
The center 3 is not visible from any direction; for it to be visible, there would need to be only trees of at most height 2 between it and an edge.
The right-middle 3 is visible from the right.
In the bottom row, the middle 5 is visible, but the 3 and 4 are not.
With 16 trees visible on the edge and another 5 visible in the interior, a total of 21 trees are visible in this arrangement.

Consider your map; how many trees are visible from outside the grid?

In [9]:
!pip install numpy

     |████████████████████████████████| 19.8 MB 1.4 MB/s eta 0:00:01


In [29]:
# first time using numpy in the advent of code!
import numpy as np

def parse_input(input_text):
    # Convert to list of list of ints
    string_rows = input_text.split("\n")
    parsed_input = []
    for string_row in string_rows:
        parsed_input.append([int(tree_height) for tree_height in string_row])
    return np.array(parsed_input)
    

# Going to make a more simple implimentation at first where I just find if each tree is visible, then 
# make a more performant version where we only look from "outside" the forest

def find_num_visible(input_text):
    parsed_input = parse_input(input_text)
    num_visible = 0

    for tree_row in range(parsed_input.shape[0]):
        for tree_col in range(parsed_input.shape[1]):
            current_tree_height = parsed_input[tree_row, tree_col]
            # The conditions state if a tree is on the edge it is visible from that direction
            tallest_trees_above = parsed_input[:tree_row,tree_col].max() if tree_row > 0 else -1
            tallest_trees_below = parsed_input[tree_row+1:,tree_col].max() if tree_row < parsed_input.shape[0]-1 else -1
            tallest_trees_left = parsed_input[tree_row,:tree_col].max() if tree_col > 0 else -1
            tallest_trees_right = parsed_input[tree_row,tree_col+1:].max() if tree_col < parsed_input.shape[1]-1 else -1
            is_visible = min(
                tallest_trees_above,
                tallest_trees_below,
                tallest_trees_left,
                tallest_trees_right
            ) < current_tree_height
            if is_visible:
                num_visible += 1
    return num_visible

find_num_visible(open("input.txt").read())

1715

--- Part Two ---
Content with the amount of tree cover available, the Elves just need to know the best spot to build their tree house: they would like to be able to see a lot of trees.

To measure the viewing distance from a given tree, look up, down, left, and right from that tree; stop if you reach an edge or at the first tree that is the same height or taller than the tree under consideration. (If a tree is right on the edge, at least one of its viewing distances will be zero.)

The Elves don't care about distant trees taller than those found by the rules above; the proposed tree house has large eaves to keep it dry, so they wouldn't be able to see higher than the tree house anyway.

In the example above, consider the middle 5 in the second row:

```
30373
25512
65332
33549
35390
```

Looking up, its view is not blocked; it can see 1 tree (of height 3).
Looking left, its view is blocked immediately; it can see only 1 tree (of height 5, right next to it).
Looking right, its view is not blocked; it can see 2 trees.
Looking down, its view is blocked eventually; it can see 2 trees (one of height 3, then the tree of height 5 that blocks its view).
A tree's scenic score is found by multiplying together its viewing distance in each of the four directions. For this tree, this is 4 (found by multiplying 1 * 1 * 2 * 2).

However, you can do even better: consider the tree of height 5 in the middle of the fourth row:

```
30373
25512
65332
33549
35390
```

Looking up, its view is blocked at 2 trees (by another tree with a height of 5).
Looking left, its view is not blocked; it can see 2 trees.
Looking down, its view is also not blocked; it can see 1 tree.
Looking right, its view is blocked at 2 trees (by a massive tree of height 9).
This tree's scenic score is 8 (2 * 2 * 1 * 2); this is the ideal spot for the tree house.

Consider each tree on your map. What is the highest scenic score possible for any tree?

In [59]:
def find_most_scenic(input_text):
    """Find the tree with the best view"""
    parsed_input = parse_input(input_text)
    most_scenic = -1
    
    for tree_row in range(parsed_input.shape[0]):
        for tree_col in range(parsed_input.shape[1]):
            current_tree_height = parsed_input[tree_row, tree_col]
            # How many trees can we see in each direction
            trees_above_taller = parsed_input[:tree_row,tree_col][::-1] >= current_tree_height
            if trees_above_taller.any():
                view_top = trees_above_taller.argmax() + 1
            else:
                view_top = max(tree_row, 1)
            if tree_row == parsed_input.shape[0]-1:
                view_below = 1
            else:
                trees_below_taller = parsed_input[tree_row+1:,tree_col] >= current_tree_height
                if trees_below_taller.any():
                    view_below = trees_below_taller.argmax() + 1
                else:
                    view_below = max(parsed_input.shape[0] - tree_row-1, 1)
            trees_left_taller = parsed_input[tree_row,:tree_col][::-1] >= current_tree_height
            if trees_left_taller.any():
                view_left = trees_left_taller.argmax() + 1
            else:
                view_left = max(tree_col, 1)
        
            if tree_col == parsed_input.shape[1]-1:
                view_right = 1
            else:
                trees_right_taller = parsed_input[tree_row,tree_col+1:] >= current_tree_height
                if trees_right_taller.any():
                    view_right = trees_right_taller.argmax() + 1
                else:
                    view_right = max(parsed_input.shape[0] - tree_col-1, 1)
            scenery_score = view_top * view_below * view_left * view_right
            if scenery_score > most_scenic:
                most_scenic = scenery_score
    return most_scenic
find_most_scenic(open("input.txt").read())

374400